In [0]:
import os

In [0]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive/"

Mounted at /content/gdrive


In [0]:
data_root_dir = "/content/gdrive/My Drive/data/"

In [0]:
reagent_data_dir = data_root_dir + 'reagent-data/'

In [0]:
os.listdir(reagent_data_dir)

['reagent_data_30092019-part3.csv',
 'reagent_data_09-10-2019.csv',
 'lab-test-data.gsheet',
 'lab-test-data.csv']

In [0]:
file_name = reagent_data_dir + 'reagent_data_30092019-part3.csv'

In [0]:
import numpy as np # linear algebra
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

# Model Building

In [0]:
from fastai import *
from fastai.tabular import *

In [0]:
df_reagent = pd.read_csv(file_name, parse_dates= True)
df_reagent['date'] = pd.to_datetime(df_reagent['date'], errors='coerce')
df_reagent.sort_values(by = "date", inplace=True)
df_reagent.drop("Unnamed: 0", axis =1, inplace = True)
add_datepart(df_reagent, "date")


from sklearn.model_selection import train_test_split

In [0]:
df_reagent.tail(10)

,reagent_name,pack_size,num_pack_units,reagent_wip_volume,qc_control_runs,qc_failure,qc_failure_repeat,qc_total_runs,test_runs,repeated_test_runs,reagent_wastage,calibration_runs_scheduled,calibration_runs_on_failure,total_calibration_runs,total_reagent_volume,protocol_in_ml,Year,Month,Week,Day,Dayofweek,Dayofyear,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,Elapsed
2893,Reagent-BNP,200,2,207.1,3,0,0,3,20,9,0.0,0,0,0,32.0,1.0,2018,12,49,6,3,340,False,False,False,False,False,False,1544054400
1923,Reagent-Anti-HAViGG,400,3,379.6,3,1,0,4,36,4,0.0,0,0,0,88.0,2.0,2018,12,49,7,4,341,False,False,False,False,False,False,1544140800
2923,Reagent-BNP,200,2,386.8,3,1,1,5,20,0,3.0,0,9,9,37.0,1.0,2018,12,49,7,4,341,False,False,False,False,False,False,1544140800
923,Reagent-AFP,300,3,446.5,3,1,0,4,33,4,0.0,0,0,0,20.5,0.5,2018,12,49,7,4,341,False,False,False,False,False,False,1544140800
2954,Reagent-BNP,200,3,85.9,3,3,0,6,20,4,0.0,0,0,0,30.0,1.0,2018,12,49,8,5,342,False,False,False,False,False,False,1544227200
1954,Reagent-Anti-HAViGG,400,2,80.2,3,0,0,3,48,7,0.0,0,0,0,116.0,2.0,2018,12,49,8,5,342,False,False,False,False,False,False,1544227200
954,Reagent-AFP,300,2,343.5,3,2,2,7,30,3,0.0,0,6,6,23.0,0.5,2018,12,49,8,5,342,False,False,False,False,False,False,1544227200
2985,Reagent-BNP,200,3,251.0,3,0,0,3,16,4,0.0,0,0,0,23.0,1.0,2018,12,49,9,6,343,False,False,False,False,False,False,1544313600
1985,Reagent-Anti-HAViGG,400,3,652.0,3,1,1,5,44,5,0.0,0,6,6,120.0,2.0,2018,12,49,9,6,343,False,False,False,False,False,False,1544313600
985,Reagent-AFP,300,3,869.0,3,0,0,3,27,7,0.0,0,0,0,18.5,0.5,2018,12,49,9,6,343,False,False,False,False,False,False,1544313600


In [0]:
df_reagent.columns

Index(['reagent_name', 'pack_size', 'num_pack_units', 'reagent_wip_volume',
       'qc_control_runs', 'qc_failure', 'qc_failure_repeat', 'qc_total_runs',
       'test_runs', 'repeated_test_runs', 'reagent_wastage',
       'calibration_runs_scheduled', 'calibration_runs_on_failure',
       'total_calibration_runs', 'total_reagent_volume', 'protocol_in_ml',
       'Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear',
       'Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start',
       'Is_year_end', 'Is_year_start', 'Elapsed'],
      dtype='object')

In [0]:
drop_columns = ['pack_size', 'num_pack_units', 'reagent_wip_volume' , 'qc_control_runs', 'qc_failure', 'qc_failure_repeat', 'qc_total_runs', 'test_runs' ]
drop_columns.append('repeated_test_runs')
drop_columns.append('reagent_wastage')
drop_columns.append('calibration_runs_scheduled')
drop_columns.append('calibration_runs_on_failure')
drop_columns.append('total_calibration_runs')
drop_columns.append('protocol_in_ml')
df_reagent.drop( drop_columns, axis =1, inplace = True);  df_reagent.head()
#df.drop(df.columns[[0, 4, 2]], axis = 1, inplace = True) 

,reagent_name,total_reagent_volume,Year,Month,Week,Day,Dayofweek,Dayofyear,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,Elapsed
0,Reagent-AFP,24.00,2016,1,53,1,4,1,False,True,False,True,False,True,1451606400
2000,Reagent-BNP,43.00,2016,1,53,1,4,1,False,True,False,True,False,True,1451606400
1000,Reagent-Anti-HAViGG,130.00,2016,1,53,1,4,1,False,True,False,True,False,True,1451606400
2031,Reagent-BNP,36.20,2016,1,53,2,5,2,False,False,False,False,False,False,1451692800
31,Reagent-AFP,30.75,2016,1,53,2,5,2,False,False,False,False,False,False,1451692800


In [0]:
df_reagent.head()

,reagent_name,total_reagent_volume,Year,Month,Week,Day,Dayofweek,Dayofyear,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,Elapsed
0,Reagent-AFP,24.00,2016,1,53,1,4,1,False,True,False,True,False,True,1451606400
2000,Reagent-BNP,43.00,2016,1,53,1,4,1,False,True,False,True,False,True,1451606400
1000,Reagent-Anti-HAViGG,130.00,2016,1,53,1,4,1,False,True,False,True,False,True,1451606400
2031,Reagent-BNP,36.20,2016,1,53,2,5,2,False,False,False,False,False,False,1451692800
31,Reagent-AFP,30.75,2016,1,53,2,5,2,False,False,False,False,False,False,1451692800


In [0]:
dep_var = 'total_reagent_volume'
cat_names = ['reagent_name']
cont_names = ['Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear', 'Elapsed', 'Is_month_end', 'Is_month_start','Is_quarter_end' ] # extend this
procs = [FillMissing, Categorify, Normalize]

In [0]:
test = TabularList.from_df(df_reagent.iloc[2701:3000].copy(), path=reagent_data_dir, cat_names=cat_names, cont_names=cont_names)

In [0]:
data = (TabularList.from_df(df_reagent, path=reagent_data_dir, cat_names=cat_names, cont_names=cont_names, procs=procs)
                           .split_by_idx(list(range(2701,3000)))
                           .label_from_df(cols=dep_var)
                           .add_test(test)
                           .databunch())

In [0]:
data.show_batch(rows=10)

reagent_name,Year,Month,Week,Day,Dayofweek,Dayofyear,Elapsed,Is_month_end,Is_month_start,Is_quarter_end,target
Reagent-BNP,0.2953,0.3007,0.1861,-0.7567,1.5018,0.2248,0.3938,-0.1856,-0.1888,-0.1060,27.0
Reagent-AFP,0.2953,-1.1708,-1.2968,-1.6574,-0.5003,-1.2966,-0.2075,-0.1856,5.2946,-0.1060,23.5
Reagent-Anti-HAViGG,0.2953,-0.5822,-0.6227,-0.1938,0.5007,-0.6033,0.0665,-0.1856,-0.1888,-0.1060,115.2
Reagent-BNP,-1.0567,0.5951,0.7254,1.6076,-1.0009,0.7351,-0.7973,-0.1856,-0.1888,-0.1060,32.0
Reagent-AFP,-1.0567,0.5951,0.4557,-1.5449,-1.0009,0.4655,-0.9039,-0.1856,-0.1888,-0.1060,16.55
Reagent-BNP,-1.0567,-1.1708,-1.2968,-1.0945,1.0013,-1.2484,-1.5813,-0.1856,-0.1888,-0.1060,24.1
Reagent-BNP,-1.0567,-0.8765,-1.0272,-1.5449,-0.5003,-1.0077,-1.4861,-0.1856,-0.1888,-0.1060,43.3
Reagent-BNP,-1.0567,1.1837,1.2646,1.2699,0.0002,1.2936,-0.5766,-0.1856,-0.1888,-0.1060,30.0
Reagent-Anti-HAViGG,0.2953,1.7723,1.8713,1.6076,1.0013,1.9002,1.0560,-0.1856,-0.1888,-0.1060,104.0
Reagent-Anti-HAViGG,-1.0567,0.3007,0.1187,-1.6574,0.5007,0.1574,-1.0257,-0.1856,5.2946,-0.1060,122.0


In [0]:
from fastai.layers import MSELossFlat
from fastai.metrics import mean_squared_error

learn = tabular_learner(data, layers=[200,100], loss_func=MSELossFlat(), metrics=mean_squared_error)
learn.fit(100, 1e-2)

epoch,train_loss,valid_loss,mean_squared_error,time
0,3391.034668,2897.825439,2897.825439,00:00
1,1469.017578,169.094650,169.094650,00:00
2,684.245483,152.123581,152.123581,00:00
3,400.434540,143.966980,143.966980,00:00
4,270.250061,150.853989,150.853989,00:00
5,219.829178,157.894043,157.894043,00:00
6,208.853546,167.569870,167.569870,00:00
7,198.467209,167.620331,167.620331,00:00
8,194.844620,148.996262,148.996262,00:00
9,187.386353,166.341110,166.341110,00:00


In [0]:
learn.fit(100, 1e-2)

epoch,train_loss,valid_loss,mean_squared_error,time
0,168.634735,192.024033,192.024033,00:00
1,169.377258,171.408813,171.408813,00:00
2,178.059174,185.191818,185.191818,00:00
3,177.938293,189.640915,189.640915,00:00
4,177.417343,195.524902,195.524902,00:00
5,167.864319,192.603607,192.603607,00:00
6,178.055817,255.818146,255.818146,00:00
7,183.304001,344.177765,344.177765,00:00
8,175.019867,962.411987,962.411987,00:00
9,180.688980,7122.094238,7122.094238,00:00


In [0]:
row = df_reagent.iloc[2999]; print("Actual:", row.total_reagent_volume, "Predicted:", learn.predict(row)[2])

Actual: 18.5 Predicted: tensor([19.3247])


In [0]:
#Display Predictions On Training Data
learn.show_results(ds_type=DatasetType.Train,rows = 5)
#Display Predictions On Validation Data


reagent_name,Year,Month,Week,Day,Dayofweek,Dayofyear,Elapsed,Is_month_end,Is_month_start,Is_quarter_end,target,prediction
Reagent-AFP,1.6473,-1.1708,-1.0946,0.9321,1.0013,-1.0751,1.2691,-0.1856,-0.1888,-0.1060,20.55,[20.756771]
Reagent-Anti-HAViGG,0.2953,0.5951,0.5231,-0.9819,-1.5014,0.5040,0.5042,-0.1856,-0.1888,-0.1060,126.0,[110.13854]
Reagent-Anti-HAViGG,-1.0567,-0.2879,-0.2183,1.4950,1.5018,-0.1604,-1.1513,-0.1856,-0.1888,-0.1060,86.0,[102.49644]
Reagent-BNP,1.6473,-1.4652,-1.5664,-1.6574,-1.5014,-1.5951,1.0636,-0.1856,5.2946,-0.1060,46.0,[27.774914]
Reagent-AFP,0.2953,1.7723,1.6690,-1.3197,-1.5014,1.6498,0.9570,-0.1856,-0.1888,-0.1060,19.6,[17.65467]


In [0]:
learn.show_results(ds_type=DatasetType.Valid, rows = 20)

reagent_name,Year,Month,Week,Day,Dayofweek,Dayofyear,Elapsed,Is_month_end,Is_month_start,Is_quarter_end,target,prediction
Reagent-BNP,1.6473,0.3007,0.1861,-0.9819,1.0013,0.2055,1.7752,-0.1856,-0.1888,-0.1060,30.6,[35.11735]
Reagent-AFP,1.6473,0.3007,0.1861,-0.9819,1.0013,0.2055,1.7752,-0.1856,-0.1888,-0.1060,14.5,[23.04623]
Reagent-BNP,1.6473,0.3007,0.1861,-0.8693,1.5018,0.2151,1.7790,-0.1856,-0.1888,-0.1060,26.2,[31.825678]
Reagent-AFP,1.6473,0.3007,0.1861,-0.8693,1.5018,0.2151,1.7790,-0.1856,-0.1888,-0.1060,28.15,[22.110996]
Reagent-Anti-HAViGG,1.6473,0.3007,0.1861,-0.8693,1.5018,0.2151,1.7790,-0.1856,-0.1888,-0.1060,135.4,[108.40148]
Reagent-BNP,1.6473,0.3007,0.2535,-0.7567,-1.5014,0.2248,1.7828,-0.1856,-0.1888,-0.1060,34.0,[37.277225]
Reagent-AFP,1.6473,0.3007,0.2535,-0.7567,-1.5014,0.2248,1.7828,-0.1856,-0.1888,-0.1060,24.7,[25.500824]
Reagent-Anti-HAViGG,1.6473,0.3007,0.2535,-0.7567,-1.5014,0.2248,1.7828,-0.1856,-0.1888,-0.1060,92.6,[109.0066]
Reagent-AFP,1.6473,0.3007,0.2535,-0.3064,0.5007,0.2633,1.7981,-0.1856,-0.1888,-0.1060,26.5,[26.908073]
Reagent-Anti-HAViGG,1.6473,0.3007,0.2535,-0.3064,0.5007,0.2633,1.7981,-0.1856,-0.1888,-0.1060,119.2,[120.08536]


In [0]:
data.train_ds

LabelList (2701 items)
x: TabularList
reagent_name Reagent-AFP; Year -1.0567; Month -1.4652; Week 1.9387; Day -1.6574; Dayofweek 0.5007; Dayofyear -1.5951; Elapsed -1.7183; Is_month_end -0.1856; Is_month_start 5.2946; Is_quarter_end -0.1060; ,reagent_name Reagent-BNP; Year -1.0567; Month -1.4652; Week 1.9387; Day -1.6574; Dayofweek 0.5007; Dayofyear -1.5951; Elapsed -1.7183; Is_month_end -0.1856; Is_month_start 5.2946; Is_quarter_end -0.1060; ,reagent_name Reagent-Anti-HAViGG; Year -1.0567; Month -1.4652; Week 1.9387; Day -1.6574; Dayofweek 0.5007; Dayofyear -1.5951; Elapsed -1.7183; Is_month_end -0.1856; Is_month_start 5.2946; Is_quarter_end -0.1060; ,reagent_name Reagent-BNP; Year -1.0567; Month -1.4652; Week 1.9387; Day -1.5449; Dayofweek 1.0013; Dayofyear -1.5854; Elapsed -1.7145; Is_month_end -0.1856; Is_month_start -0.1888; Is_quarter_end -0.1060; ,reagent_name Reagent-AFP; Year -1.0567; Month -1.4652; Week 1.9387; Day -1.5449; Dayofweek 1.0013; Dayofyear -1.5854; Elapsed -1.7145

In [0]:
data.valid_ds


LabelList (299 items)
x: TabularList
reagent_name Reagent-BNP; Year 1.6473; Month 0.3007; Week 0.1861; Day -0.9819; Dayofweek 1.0013; Dayofyear 0.2055; Elapsed 1.7752; Is_month_end -0.1856; Is_month_start -0.1888; Is_quarter_end -0.1060; ,reagent_name Reagent-AFP; Year 1.6473; Month 0.3007; Week 0.1861; Day -0.9819; Dayofweek 1.0013; Dayofyear 0.2055; Elapsed 1.7752; Is_month_end -0.1856; Is_month_start -0.1888; Is_quarter_end -0.1060; ,reagent_name Reagent-BNP; Year 1.6473; Month 0.3007; Week 0.1861; Day -0.8693; Dayofweek 1.5018; Dayofyear 0.2151; Elapsed 1.7790; Is_month_end -0.1856; Is_month_start -0.1888; Is_quarter_end -0.1060; ,reagent_name Reagent-AFP; Year 1.6473; Month 0.3007; Week 0.1861; Day -0.8693; Dayofweek 1.5018; Dayofyear 0.2151; Elapsed 1.7790; Is_month_end -0.1856; Is_month_start -0.1888; Is_quarter_end -0.1060; ,reagent_name Reagent-Anti-HAViGG; Year 1.6473; Month 0.3007; Week 0.1861; Day -0.8693; Dayofweek 1.5018; Dayofyear 0.2151; Elapsed 1.7790; Is_month_end -0.

In [0]:
data.test_ds

LabelList (299 items)
x: TabularList
reagent_name Reagent-BNP; Year 1.6473; Month 0.3007; Week 0.1861; Day -0.9819; Dayofweek 1.0013; Dayofyear 0.2055; Elapsed 1.7752; Is_month_end -0.1856; Is_month_start -0.1888; Is_quarter_end -0.1060; ,reagent_name Reagent-AFP; Year 1.6473; Month 0.3007; Week 0.1861; Day -0.9819; Dayofweek 1.0013; Dayofyear 0.2055; Elapsed 1.7752; Is_month_end -0.1856; Is_month_start -0.1888; Is_quarter_end -0.1060; ,reagent_name Reagent-BNP; Year 1.6473; Month 0.3007; Week 0.1861; Day -0.8693; Dayofweek 1.5018; Dayofyear 0.2151; Elapsed 1.7790; Is_month_end -0.1856; Is_month_start -0.1888; Is_quarter_end -0.1060; ,reagent_name Reagent-AFP; Year 1.6473; Month 0.3007; Week 0.1861; Day -0.8693; Dayofweek 1.5018; Dayofyear 0.2151; Elapsed 1.7790; Is_month_end -0.1856; Is_month_start -0.1888; Is_quarter_end -0.1060; ,reagent_name Reagent-Anti-HAViGG; Year 1.6473; Month 0.3007; Week 0.1861; Day -0.8693; Dayofweek 1.5018; Dayofyear 0.2151; Elapsed 1.7790; Is_month_end -0.

In [0]:
df_reagent.head()

,reagent_name,total_reagent_volume,Year,Month,Week,Day,Dayofweek,Dayofyear,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,Elapsed
0,Reagent-AFP,24.00,2016,1,53,1,4,1,False,True,False,True,False,True,1451606400
2000,Reagent-BNP,43.00,2016,1,53,1,4,1,False,True,False,True,False,True,1451606400
1000,Reagent-Anti-HAViGG,130.00,2016,1,53,1,4,1,False,True,False,True,False,True,1451606400
2031,Reagent-BNP,36.20,2016,1,53,2,5,2,False,False,False,False,False,False,1451692800
31,Reagent-AFP,30.75,2016,1,53,2,5,2,False,False,False,False,False,False,1451692800


In [0]:
train_last_index = 2701
df_reagent[train_last_index : train_last_index + 5]
df_valid = df_reagent[train_last_index:]

In [0]:
learn.show_results(ds_type=DatasetType.Valid, rows = 5)

reagent_name,Year,Month,Week,Day,Dayofweek,Dayofyear,Elapsed,Is_month_end,Is_month_start,Is_quarter_end,target,prediction
Reagent-BNP,1.6473,0.3007,0.1861,-0.9819,1.0013,0.2055,1.7752,-0.1856,-0.1888,-0.1060,30.6,[31.41616]
Reagent-AFP,1.6473,0.3007,0.1861,-0.9819,1.0013,0.2055,1.7752,-0.1856,-0.1888,-0.1060,14.5,[17.741074]
Reagent-BNP,1.6473,0.3007,0.1861,-0.8693,1.5018,0.2151,1.7790,-0.1856,-0.1888,-0.1060,26.2,[28.112972]
Reagent-AFP,1.6473,0.3007,0.1861,-0.8693,1.5018,0.2151,1.7790,-0.1856,-0.1888,-0.1060,28.15,[17.233162]
Reagent-Anti-HAViGG,1.6473,0.3007,0.1861,-0.8693,1.5018,0.2151,1.7790,-0.1856,-0.1888,-0.1060,135.4,[105.68922]


In [0]:
print(df_valid[0:10].reagent_name , df_valid[0:10].total_reagent_volume)

2918            Reagent-BNP
918             Reagent-AFP
2949            Reagent-BNP
949             Reagent-AFP
1949    Reagent-Anti-HAViGG
2980            Reagent-BNP
980             Reagent-AFP
1980    Reagent-Anti-HAViGG
924             Reagent-AFP
1924    Reagent-Anti-HAViGG
Name: reagent_name, dtype: object 2918     30.60
918      14.50
2949     26.20
949      28.15
1949    135.40
2980     34.00
980      24.70
1980     92.60
924      26.50
1924    119.20
Name: total_reagent_volume, dtype: float64
